In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql

from adam.stk import STKConfig
from adam.stk import convertPointingsToVectorInterval
from adam.stk import readSTKOutputIntoDatabase
from adam.stk import readPointingsIntoDatabase

In [2]:
# Assumes reduced file is in notebook directory
pointingFile = "reduced_astro-lsst-01_2023_19year.txt"
pointings = pd.read_csv(pointingFile, sep=" ") 

In [3]:
pointings.head()

,observationId,fieldId,fieldRA,fieldDec,altitude,azimuth,filter,night,observationStartTime,observationStartMJD,visitTime,visitExposureTime,seeingFwhmGeom,seeingFwhmEff,fiveSigmaDepth
0,1,1545,305.088793,-24.889283,84.644583,0.622416,z,1,1.664584e+09,59853.016794,34.0,30.0,0.620695,0.691843,23.348592
1,2,3968,302.892080,29.966322,29.742406,357.509430,z,1,1.664584e+09,59853.017801,34.0,30.0,0.915907,1.050982,22.545454
2,3,300,305.482200,-62.802603,57.439197,180.541142,z,1,1.664584e+09,59853.019826,34.0,30.0,0.685474,0.770650,23.167857
3,4,356,307.103419,-60.072224,60.167079,179.176566,z,1,1.664584e+09,59853.020278,34.0,30.0,0.674595,0.757415,23.201599
4,5,421,308.429608,-57.316612,62.892592,177.648233,z,1,1.664584e+09,59853.020729,34.0,30.0,0.665015,0.745760,23.231315


#### Convert telescope pointings to Vector and Interval files required by STK

In [4]:
vectorFileName = "VectorFile_LSST_19year"
intervalFileName = "IntervalList_LSST_19_year"
convertPointingsToVectorInterval(vectorFileName,
                                 intervalFileName,
                                 pointings["observationId"].values,
                                 pointings["observationStartTime"].values,
                                 pointings["fieldRA"].values,
                                 pointings["fieldDec"].values,
                                 "1 Oct 2022 00:00:00",
                                 exposureLength=30)

Writing vector pointing file: VectorFile_LSST_19year.vd
Writing observation ID to vector mapping file: VectorFile_LSST_19year_mapping.txt
Done. Wrote 9003170 pointing start and end times.

Writing interval file: IntervalList_LSST_19_year.int
Done. Wrote 4501585 intervals.



In [5]:
! head -15 {vectorFileName + ".vd"}

stk.v.11.1
Begin VectorData
NumberOfVectorDataPoints 9003170
ScenarioEpoch 1 Oct 2022 00:00:00
InterpolationSamplesM1 0
CentralBody Earth
CoordinateAxes ICRF
DimensionName Distance
VectorDataTimeRaDecMag
1664583851.177124 305.088793 -24.889283 1.0
1664583881.177124 305.088793 -24.889283 1.0
1664583938.545173 302.89207999999996 29.966322000000005 1.0
1664583968.545173 302.89207999999996 29.966322000000005 1.0
1664584113.763164 305.4822 -62.802603000000005 1.0
1664584143.763164 305.4822 -62.802603000000005 1.0


In [6]:
! head -15 {intervalFileName + ".int"}

stk.v.11.1
BEGIN IntervalList

	ScenarioEpoch 1 Oct 2022 00:00:00

Begin Intervals

1664583851.177124 1664583881.177124
1664583938.545173 1664583968.545173
1664584113.763164 1664584143.763164
1664584152.5291631 1664584182.5291631
1664584191.2933 1664584221.2933
1664584230.0539038 1664584260.0539038
1664584268.8086631 1664584298.8086631
1664584307.652108 1664584337.652108


In [7]:
! head -15 {vectorFileName + "_mapping.txt"}

1
1
2
2
3
3
4
4
5
5
6
6
7
7
8


#### Read telescope pointings into a database

In [8]:
# Stores how the pointing file columns map to ADAM internal columns
STKConfig.visitTableMapping

{'FWHMeff_arcsec': 'seeingFwhmEff',
 'FWHMgeom_arcsec': 'seeingFwhmGeom',
 'altitude_deg': 'altitude',
 'azimuth_deg': 'azimuth',
 'expDate_sec': 'observationStartTime',
 'expMJD_mjd': 'observationStartMJD',
 'fieldDec_deg': 'fieldDec',
 'fieldRA_deg': 'fieldRA',
 'field_ID': 'fieldId',
 'filter': 'filter',
 'fiveSigmaDepth_mag': 'fiveSigmaDepth',
 'night': 'night',
 'visitExpTime_sec': 'visitExposureTime',
 'visitTime_sec': 'visitTime',
 'visit_ID': 'observationId'}

In [9]:
con = sql.connect("lsst_19year_test.db")

In [10]:
readPointingsIntoDatabase(pointingFile,
                          con,
                          readKwargs={"sep" : " "},
                          columnMapping=STKConfig.visitTableMapping)

visits table doesn't exist yet. Creating table...
Reading pointing file...
Selecting necessary columns...
Mapping columns to table schema...
Saving pointings to visits table...
Done.


In [11]:
pd.read_sql("""SELECT * FROM visits LIMIT 10""", con)

,visit_ID,field_ID,fieldRA_deg,fieldDec_deg,altitude_deg,azimuth_deg,filter,expDate_sec,expMJD_mjd,night,visitTime_sec,visitExpTime_sec,FWHMgeom_arcsec,FWHMeff_arcsec,fiveSigmaDepth_mag
0,1,1545,305.088793,-24.889283,84.644583,0.622416,z,1.664584e+09,59853.016794,1,34.0,30.0,0.620695,0.691843,23.348592
1,2,3968,302.892080,29.966322,29.742406,357.509430,z,1.664584e+09,59853.017801,1,34.0,30.0,0.915907,1.050982,22.545454
2,3,300,305.482200,-62.802603,57.439197,180.541142,z,1.664584e+09,59853.019826,1,34.0,30.0,0.685474,0.770650,23.167857
3,4,356,307.103419,-60.072224,60.167079,179.176566,z,1.664584e+09,59853.020278,1,34.0,30.0,0.674595,0.757415,23.201599
4,5,421,308.429608,-57.316612,62.892592,177.648233,z,1.664584e+09,59853.020729,1,34.0,30.0,0.665015,0.745760,23.231315
5,6,491,309.524342,-54.543202,65.611002,175.901702,z,1.664584e+09,59853.021181,1,34.0,30.0,0.656664,0.735601,23.257295
6,7,562,310.436948,-51.758333,68.315393,173.844509,z,1.664584e+09,59853.021620,1,34.0,30.0,0.649441,0.726814,23.280534
7,8,650,311.207033,-48.966530,70.997948,171.351484,z,1.664584e+09,59853.022072,1,34.0,30.0,0.643267,0.719303,23.300256
8,9,735,311.867221,-46.169774,73.646761,168.187109,z,1.664584e+09,59853.022523,1,34.0,30.0,0.638079,0.712991,23.315990
9,10,2811,344.225787,3.595779,41.365948,52.399855,z,1.664584e+09,59853.024086,1,34.0,30.0,0.790241,0.898104,22.862226


#### Read STK output into a database

In [ ]:
### Need an output file that corresponds to the 19 year run otherwise this would work
objectId = "Test_object"
mappingFile = vectorFileName + "_mapping.txt"
readSTKOutputIntoDatabase(stkOutputFile,
                          mappingFile,
                          objectId,
                          con)